# Улучшение сходимости нейросетей и борьба с переобучением.

##Блок вспомогательного кода

В этом блоке определенна нейросеть и код для ее обучения который будет использоваться для демонстрации в ходе семинара. Так же он потребуется при выполнении практических заданий.

### Подготовка Tensorboard

In [ ]:
# Вспомогательный метод для запуска Tensorboard в Colab

# Fix: https://stackoverflow.com/questions/60730544/tensorboard-colab-tensorflow-api-v1-io-gfile-has-no-attribute-get-filesystem
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

import os
import shutil

# Запуск Tensorboard в Colab
def reinit_tensorboard(clear_log = True):
  # Лог-файлы читаются из этого каталога: 
  logs_base_dir = "runs"
  if clear_log:
    # Очистка логов
    #!rm -rfv {logs_base_dir}/*
    shutil.rmtree(logs_base_dir, ignore_errors = True)
    os.makedirs(logs_base_dir, exist_ok=True)
  # Магия Colab
  %load_ext tensorboard
  %tensorboard --logdir {logs_base_dir}

In [ ]:
#reinit_tensorboard(clear_log=True)

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

### Загрузка датасета MNIST

In [ ]:
%%bash
if test -f ./datasets/MNIST.tar.gz; then
    echo "Already downloaded"; 
  else 
    wget -P ./datasets www.di.ens.fr/~lelarge/MNIST.tar.gz;
    tar -zxvf ./datasets/MNIST.tar.gz -C ./datasets; 
fi

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))
     ])

trainset = torchvision.datasets.MNIST(root='./datasets', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.MNIST(root='./datasets', train=False,
                                       download=True, transform=transform)


### Класс для создания полносвязанной сети

Конструктор класса получает на воход два параметра количество слоев и функцию активации.

In [ ]:
class SimpleMNIST_NN(nn.Module):
    def __init__(self, n_layers, activation=nn.Sigmoid):
        super().__init__()
        self.n_layers = n_layers
        self.activation = activation()
        layers = [nn.Linear(28 * 28, 100), self.activation] # first layer
        for _ in range(0, n_layers - 1):
            layers.append(nn.Linear(100, 100)) # All intermediate has the same size
            layers.append(self.activation)
        layers.append(nn.Linear(100, 10))
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x, predict_proba=False):
        x = x.view(-1, 28 * 28)
        x = self.layers(x)
        if predict_proba:
          x = F.softmax(x)
        return x 

### Вспомогательные методы для обучения.

Используйте их при выполнении практической работы.

In [ ]:
train_loader = torch.utils.data.DataLoader(trainset,
                                           batch_size=32,
                                           shuffle=True, 
                                           num_workers=2)
test_loader = torch.utils.data.DataLoader(testset,
                                         batch_size=32,
                                         shuffle=False,
                                         num_workers=2)

In [ ]:
class Steper():
  def __init__(self):
    self._global_step = 1
  @property
  def global_step(self):
    return self._global_step

  def step(self):
    self._global_step += 1

import os 
def get_summary_writer(name, runs_dir="runs/"):
    path = os.path.join(runs_dir, name)
    return SummaryWriter(path)

In [ ]:
def train_epoch(model, 
                optimizer,
                criterion, 
                train_loader, 
                model_name,
                steper, 
                writer):
    for batch in train_loader:
        optimizer.zero_grad()
        X, y = batch
        X, y = X.cuda(), y.cuda()
        pred = model(X)
        loss = criterion(pred, y)
        writer.add_scalar(f"{model_name}_train_loss", 
                          loss.cpu().detach().numpy(),
                          global_step= steper.global_step)
        loss.backward()
        optimizer.step()
        steper.step()


In [ ]:
def validate(model, 
             validation_loader,
             model_name,
             steper,
             writer):
    cumloss = 0
    with torch.no_grad():
        for batch in validation_loader:
            X, y = batch
            X, y = X.cuda(), y.cuda()
            pred = model(X)
            loss = criterion(pred, y)
            writer.add_scalar(f"{model_name}_test_loss", 
                          loss.cpu().detach().numpy(),
                          global_step= steper.global_step)
            steper.step()
            cumloss += loss
    return cumloss / len(validation_loader)



## Нормализация входных данных
В 5й лекции мы убедились что нормализация данных оказывает заметное влияние на сходимость модели. Давайте разберемся почему так происходит.


1. Масштаб. Для табличных данных это было очевидно. И хотя пикселы изображения ближе по абсолютной величине друг к другу для них масштабирование тоже может быть актуально.

2. Стабильность. Малые изменения весов меньше влияют на результат. 

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/norm_stab.png" width="700">

3. Скорость обучения

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/norm_speed.png" width="500">

4. Активации

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/norm_activations.png" width="500">



Представим себе, что данные, которые мы подаем в нейросеть, распределены следующщим образом

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/no_normalization.png" width="300">


Фактически нейросети работают со скалярными произведениями. В этом плане две вектора, изображенных на рисунке, не сильно отличаются. Так же и точки нашего датасета слабо разделимы. Чтобы с этим работать, нейросеть сначала должна подобрать удобное преобразование, а затем только сравнивать наши объекты. Понятно, это усложняют задачу. 

Для того, чтобы облегчить нейросети задачу, входные признаки часто нормируют:

$$x1' = \dfrac {x1 - \mu_{x1}} {\sigma_{x1}}$$
$$x2' = \dfrac {x2 - \mu_{x2}} {\sigma_{x2}}$$

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/normalization.png" width="300">


 Такое преобразование действительно помогает нейросети 
 




###В чем состоит нормализация:

Вычитаем среднее, делеим на стандартное отклонение.

Для табличных данных иногда применяют PCA а затем снова нормализуют. Но для изображений такая техника не применяется (Почему?)

 <img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/norm_pca.png" width="700">

Нормализация изображений в Pytorch 

In [ ]:
torchvision.transforms.ToTensor() # x /= 255 -> [0 .. 1]
# per channel mean, std
torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],[0.2470, 0.2434, 0.2615]) 
                                            

## Инициализация весов

То что нормализация входных данных полезна мы  убедились при работе на неглубоких моделях.

Но что будет если в модели несколько слоев?

Входом каждого следующего слоя буде выход предшествующего.

А его значения зависят как от входа так и от весов.

 <img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/weights_nn.png" width="700">

 Давайте посмотрим на распределение выходов внутренних словев.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (20,12)

def show_act(ax,x):
  ax.hist(x, bins=500, range=(-3,3))
  std = x.std()
  ax.title.set_text(f"std {std:.9f}")

# Model from forth lecture with added third layer
class NeuralNet:
  def __init__(self):
    self.W1 = np.random.randn(3072, 500) * 0.001 
    self.W2 = np.random.randn(500, 100) * 0.1  # 0.1 10
    self.W3 = np.random.randn(100, 10) * 0.5 # 0.5 100
   

  def predict(self,x):

    fig, axs = plt.subplots(4, 1)
    fig.tight_layout()
    show_act(axs[0],x)

    # Layer 1
    scores1 = x.dot(self.W1) # Linear / Fully connected layer1
    activations1 = np.tanh(scores1) # ReLU activation
    show_act(axs[1],activations1)
    
    # Layer 2
    scores2 = activations1.dot(self.W2)
    activations2 = np.tanh(scores2) # ReLU activation
    show_act(axs[2],activations2)
    
    # Layer 3
    scores3 = activations2.dot(self.W3)
    show_act(axs[3],scores3)

    return scores3

x = np.random.randn(3072) # input image
simple_net = NeuralNet()
scores = simple_net.predict(x)
#print(scores)

Выходы схлопаваются.


Очевидно это связанно с весами. Кстати почему мы вообще так их инициализируем.

Почему бы не использовать ноль ..

или 1- цу
...

или сделать их побольше, мы же все время на них умножаем, а выходы и так маленикие.

.. 

похоже что это работает если на каждом слое масштабировать веса, то коллапса можно избежать...


Хотим задать изначальные веса в нейросети. 
Как это можно сделать? 

Делать инициализацию 0 - плохая идея. В этом случае нейросеть получим, что градиент по всем таким весам будет одинаков - и их обновление, следовательно, тоже. Ничего не выучим. 

Можно инициализировать веса нормальным шумом с матожиданием 0 и маленькой дисперсией. Маленькая дисперсия нужна затем, чтобы не получить огромные градиенты за большие изначальные ошибки в предсказании. 

Но почему бы не взять другое распределение? Может, веса, инициализированные экспоненциальным распределениям дают какие-нибудь классные гарантии? Или какой-нибудь другой, не приходящий сразу в голову способ инициализации. Давайте это обсудим. 


### Эксперимент: 2Layer vs 3Layer
(Сигмоида затухает и теоретически и практически)

In [ ]:
model = SimpleMNIST_NN(n_layers=2).cuda()
optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss().cuda()
writer = get_summary_writer("n_layers2_sigmoid")

train_steper = Steper()
test_steper = Steper()

In [ ]:

for epoch in range(10):
    train_epoch(model,
              optimizer, 
              criterion,
              train_loader,
              "cross_entropy",
              train_steper,
              writer)
    validate(model, 
           test_loader, 
           "cross_entropy", 
           test_steper,
           writer)
    


In [ ]:
from torch import nn
model = SimpleMNIST_NN(n_layers=3).cuda()
optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss().cuda()
writer = get_summary_writer("n_3layers_sigmoid")

train_steper = Steper()
test_steper = Steper()

In [ ]:
for epoch in range(10):
    train_epoch(model,
              optimizer, 
              criterion,
              train_loader,
              "cross_entropy",
              train_steper,
              writer)
    validate(model, 
           test_loader, 
           "cross_entropy", 
           test_steper,
           writer)

In [ ]:
reinit_tensorboard(clear_log=False)

Нейросеть с тремя слоями вообще не учится. Почему? Можем попробовать разобраться

Для этого напишем функции, которые будут следить за распределением градиентов и активаций на наших слоях.

И скажем при помощи метода register_backward_hook  пайторчу исполнять эти функции при каждом пропускании градиента

In [ ]:
def get_forward_hook(writer, tag):
    ind = 0
    def forward_hook(self, input_, output):
        nonlocal ind
        ind += 1
        writer.add_scalar(tag,
                          input_[0].abs().mean(),
                          ind)
    return forward_hook

def get_backward_hook(writer, tag):
    ind = 0
    def backward_hook(grad): # for tensors 
        nonlocal ind
        ind += 1
        writer.add_scalar(tag, 
                          grad.abs().mean(), 
                          ind)
    return backward_hook



In [ ]:
def register_model_hooks(model, writer, max_ind = 4):
  cur_ind = 0
  for child in model.layers.children():
      if isinstance(child, nn.Linear):
          cur_ind += 1
          forward_hook = get_forward_hook(writer, f"activation_{cur_ind}")
          child.register_forward_hook(forward_hook)

          backward_hook = get_backward_hook(writer, f"gradient_{max_ind - cur_ind + 1}")
          child.weight.register_hook(backward_hook)

In [ ]:
model = SimpleMNIST_NN(n_layers=3).cuda()
optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss().cuda()
writer = get_summary_writer("n3_layers_sigmoid2")

train_steper = Steper()
test_steper = Steper()

In [ ]:
register_model_hooks(model, writer)

In [ ]:
for epoch in range(10):
    train_epoch(model,
              optimizer, 
              criterion,
              train_loader,
              "cross_entropy",
              train_steper,
              writer)
    validate(model, 
           test_loader, 
           "cross_entropy", 
           test_steper,
           writer)

In [ ]:
reinit_tensorboard(clear_log=False)

Мы видим, что градиент нашей модели стремительно затухает. Первые слои (до которых градиент доходит последним), получают значения градиента, мало отличимые от нуля

Причем, это будет верно с самых первых шагов обучения нашей модели



Это явление получило название **паралич сети**

Оказывается, иногда чтобы побороть такое поведение нейросети достаточно лишь правильно инициализировать веса 

### Инициализация Ксавьера (Xavier, Glorot)

Рассмотрим в качестве активации нечетную функцию с единичной производной в нуле .



Например, нам подойдет гиперболический тангенс (tanh)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.arange(-10, 10.1, 0.1)
plt.plot(x, np.tanh(x))


Мы хотим начать из линейного региона этой функции, чтобы избежать
затухающих градиентов

Как у нас зависят активации на текущем слое от активаций на предыдущем?

$$z^{i+1} = f(z^iW^i)$$

Тогда, так как мы вначале хотим находиться в районе линейности нашей функции, то

$$z^{i+1} \approx z^i W^i$$

Ответить на вопрос, а как будут связаны дисперсии этих активаций сложнее 

Сначала распишем:
1. чему равна дисперсия суммы двух независимых величин

$$D(\eta + \gamma) = D\eta + D\gamma$$

2. чему равна дисперсия произведений двух независимых величин


$$D\eta\gamma = E(\eta\gamma)^2 - (E\eta\gamma)^2 = E\eta^2E\gamma^2 - (E\eta)^2(E\gamma)^2$$ 

Далее распишем для одного веса текущего слоя 

$$z^{i+1}_{k} = \sum_t z^i_t w_{kt}$$

$$D(z^{i+1}_{k}) = D(\sum_t z^i_t w_{kt}) = \sum_t D(z^i_t w_{kt})$$

Предполагая, что дисперсии весов и активаций одинаковые (а нам бы так хотелось)

$$D(z^{i+1}_{k}) = n D(z^i_0 w_{k0})$$

Далее применяем нашу формулу и получаем:

$$D(z^{i+1}_{k}) = n [E(z^i_0)^2E(w_{k0})^2 - (Ez^i_0)^2(Ew_{k0})^2]$$

Хотим, чтобы что матожидание наших активаций 0, и мы можем этого добиться, делая  матожидание весов равным 0. 

$$D(z^{i+1}_{k}) =   n E(z^i_0)^2E(w_{k0})^2 $$

Заметим, что так как матожидание активация и весов равны 0, то матожидания их квадратов равны дисперсии активаций и весов соответственно

$$Dz = E(z^{i+1}_{k})^2 - (Ez^{i+1}_{k})^2 = E(z^{i+1}_{k})^2$$

$$D(z^{i+1}_{k}) = n Dz^i_0Dw_{k0}$$




Отсюда можно вывести формулу для зависимости активаций любого слоя от весов предуыдущих слоев и дисперсии исходных данных

$$Dz^i = Dx \prod_{p=0}^{i-1}n_pDW^p $$

Где n_p - размерность выхода слоя p-го слоя

Аналогично можно вывести формулу для градиентов по активациям


$$D(\dfrac {\delta L} {\delta z^i}) = D(\dfrac {\delta L} {\delta z^d} ) \prod_{p=i}^{d}n_{p+1}DW^p $$



Для хорошей инициализации нам бы хотелось, чтобы дисперсии активаций на всех слоях были одинаковые. 
А еще хотелось бы, чтобы дисперсии градиентов тоже были одинаковые. 

Тогда у нас не происходит резких скачков в распределении активаций, а градиент не затухает и не взрывается 


$$Dz^i = Dz^j$$
$$D\dfrac {\delta L} {\delta z^i} = D\dfrac {\delta L} {\delta z^j}$$


Учитывая предыдущее, это эквивалентно тому, что мы требуем

$$n_iDW^i = 1$$
$$n_{i+1}DW^i = 1$$

Одновременно так сделать не получится $$n_i \ne n_{i+1}$$

Потому делаем компромисс - среднее гармонческое решений первого и второго уравнения

$$DW^i = \dfrac 2 {n_i + n_{i+1}}$$

Надо выбрать распределение.

$$ EW^i = 0 $$

$$DW^i = \dfrac 2 {n_i + n_{i+1}}$$

Можно было бы взять нормальное распределение с такими параметрами.

$$W_i \sim N(0, sd=\sqrt{\dfrac 2 {n_i + n_{i+1}}}) $$

А можно равномерное:

$$D(U[a, b]) = \dfrac 1 {12} (b -a)^2$$

$$W_i \sim U[-\dfrac {\sqrt{6}} {n_i + n_{i + 1}}, \dfrac {\sqrt{6}} {n_i + n_{i + 1}} ]$$



In [ ]:
import math
class X_NeuralNet(NeuralNet):
  def __init__(self):
    super().__init__()
    self.W1 = np.random.randn(3072, 500) * (1 / np.sqrt(3072))  
    self.W2 = np.random.randn(500, 100) * (1 / np.sqrt(500)) 
    self.W3 = np.random.randn(100, 10) * (1  / np.sqrt(100))

    # Rule
    # W = np.random.randn(Din,Dout) / np.sqrt(Din)

simple_net = X_NeuralNet()
scores = simple_net.predict(x)

### He-инициализация (Kaiming)

Но многие люди используют ReLU. Она в 0 не имеет производной, потому применять инициализацию, указанную выше, проблематично. 

Тогда получатся похожие условия 

$$Dz^i = Dx \prod_{p=0}^{i-1}\dfrac 1 2 n_pDW^p $$

$$D(\dfrac {\delta L} {\delta z^i}) = D(\dfrac {\delta L} {\delta z^d} ) \prod_{p=i}^{d}\dfrac 1 2 n_{p+1}DW^p $$


И для них решения будут тоже похожими:

$$\frac 2 {n_k}$$

и

$$\frac 2 {n_{k+1}}$$

Можно опять взять среднее гармоническое. Но на практике, особенно в случае сверточных нейронных сетей, просто берут либо $ \frac 2 {n_i}$ либо $\frac 2 {n_i + 1}$

$$W^i \sim N(0, sd=\sqrt{\frac 2 n_i})$$

$$W^i \sim N(0, sd=\sqrt{\frac 2 {n_i + 1}})$$

Опять же, можно использовать и равномерное распределение

In [ ]:
# Rule
# W = np.random.randn(Din,Dout) / np.sqrt(2/Din)

### Важность соответствия функции активации выбранной инициализации

От правильной активации может зависеть очень многое:

1. Нейросеть может сойтись значительно быстрее

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-07.png" width="500">

2. В зависимости от выбранной активации сеть вообще может сойтись или не сойтись

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-08.png" width="500">

### Обобщение инициализаций Ксавьера и He-инициализации

Вообще говоря, коэффициенты в инициализациях (числитель в формуле для дисперсии), зависит от конкретной выбранной функции активации.
В pytorch есть функции для вычисления этих коэффициентов
https://pytorch.org/docs/stable/nn.init.html


https://pytorch.org/docs/stable/_modules/torch/nn/modules/conv.html#Conv2d

https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html#Linear


```
init.kaiming_uniform_(self.weight, a=math.sqrt(5))
```





### Ортогональная инициализация

Также иногда используется так называемая ортогональная инициализация весов. 
Для каждого слоя мы убеждаемся, что изначальная матрица весов является ортогональной. 

Выберем ортогональную матрицу весов 
$$W: WW^T = 1$$. 

Тогда:
1.  норма активации сохраняется (опять же, активации между слоями остаются в одном масштабе)
$$||s_{i+1}|| = ||W_{i}s_i|| = ||s_i||$$

2.  все нейроны делают «разные» преобразования
$$ ⟨W_i, W_j⟩ = 0~i \ne j$$
$$ ⟨W_i, W_j⟩ = 1~i = j$$


Иногда такая инициализация обеспечивает значительно лучшую сходимость https://datascience.stackexchange.com/questions/64899/why-is-orthogonal-weights-initialization-so-important-for-ppo

https://smerity.com/articles/2016/orthogonal_init.html

https://arxiv.org/abs/2001.05992

А градиент по весам уменьшается в a раз

$$\dfrac {\delta BN((aW)u)} {\delta aW} = \dfrac 1 a \dfrac {\delta BN(Wu)} {\delta W} $$

Таким образом нейросеть автоматически не дает большим весам расти

Допустим, мы решили увеличить веса в $a$ раз

Так как мы шкалируем, то домножение весов W на константу выходных значений слоя не меняет

$$BN((aW)u) = BN(Wu)$$

Градиент слоя по входу не меняется

$$\dfrac {\delta BN((aW)u)} {\delta u} = \dfrac {\delta BN(Wu)} {\delta u}$$

### Инициализация весов в Pytorch

Для инициализации весов Pytorch используется модуль torch.nn.init

В нем определены разные функции для инициализации весов.

Нюанс состоит в том, что обычно для слоев разного типа может требоваться разная регуляриация. Потому в функции, которая инициализирует слов вашей нейронное сети желательно прописывать разное поведение для разных слоев.

Попробуем, например, добавить в нашу нейросеть инициализацию. Нам нужна регуляризация Xavier, так как у нас Sigmoid

In [ ]:
class SimpleMNIST_NN(nn.Module):
    def __init__(self, 
                 n_layers,
                 activation=nn.Sigmoid, 
                 init_form="normal"):
        super().__init__()
        self.n_layers = n_layers
        self.activation = activation()
        layers = [nn.Linear(28 * 28, 100), self.activation]
        for _ in range(0, n_layers - 1):
            layers.append(nn.Linear(100, 100))
            layers.append(self.activation)
        layers.append(nn.Linear(100, 10))
        self.layers = nn.Sequential(*layers)
        self.init_form = init_form
        if self.init_form is not None:
            self.init()
        if not isinstance(self.activation, nn.Sigmoid):
            raise NotImplementedError()
        
    def forward(self, x, predict_proba=False):
        x = x.view(-1, 28 * 28)
        x = self.layers(x)
        if predict_proba:
          x = F.softmax(x)
        return x 

    def init(self):
        sigmoid_gain = torch.nn.init.calculate_gain("sigmoid")
        for child in self.layers.children():
            if isinstance(child, nn.Linear):
                if self.init_form == "normal":
                  torch.nn.init.xavier_normal_(child.weight, 
                                                gain=sigmoid_gain)
                  if child.bias is not None:
                      torch.nn.init.zeros_(child.bias)
                elif self.init_form == "uniform":
                    torch.nn.init.xavier_uniform_(child.weight, 
                                                gain=sigmoid_gain)
                    if child.bias is not None:
                        torch.nn.init.zeros_(child.bias)
                else:
                    raise NotImplementedError()

In [ ]:
model = SimpleMNIST_NN(n_layers=3).cuda()
optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = get_summary_writer("n3_layers_sigmoid_havier")

In [ ]:
plt.hist(list(model.layers.children())[0].weight.cpu().detach().numpy().reshape(-1))
plt.show()

In [ ]:
register_model_hooks(model, writer)

In [ ]:
train_steper = Steper()
test_steper = Steper()

In [ ]:
for epoch in range(10):
    train_epoch(model,
              optimizer, 
              criterion,
              train_loader,
              "cross_entropy",
              train_steper,
              writer)
    validate(model, 
           test_loader, 
           "cross_entropy", 
           test_steper,
           writer)

In [ ]:
reinit_tensorboard(clear_log = False)

Видим, что нейросеть стала хоть как-то учиться

## Затухание(Vanishing)/Взрыв(Exploding) градиента

### Причины? 

Посмотрим на обычную сигмоиду

$$\sigma(z) = \dfrac 1 {1 + e^{-z}}$$

Ее производная, как вы уже выводили, равна

$$\dfrac {\delta \sigma(z)} {\delta z} = \sigma(z) (1 - \sigma(z))$$

Какое максимальное значение у такой функции?

Учтем, что сигмоида находится в пределах от 0 до 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

x = np.arange(0, 1.001, 0.001)
plt.plot(x, x - x**2)

Получается, что максимальное значение производной по сигмоиде - 1/4

Теперь возьмем простую нейронную сеть 

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/simple_nn.png" width="700">

Посчитаем у нее градиент

$$\dfrac {\delta L} {\delta z_4} = \dfrac {\delta L} {\delta y} \dfrac {\delta y} {\delta z_4} = \dfrac {\delta L} {\delta y} \dfrac {\delta \sigma(w_5z)} {\delta z} w_5 \le \dfrac 1 4 \dfrac {\delta L} {\delta y}  w_5 $$

Аналогично можно посчитать градиент для $z_3$

$$\dfrac {\delta L} {\delta z_3} = \dfrac {\delta L} {\delta z_4} \dfrac {\delta z_4} {\delta z_3} \le \dfrac {\delta L} {\delta y} \dfrac {\delta \sigma(w_5z)} {\delta z} w_5 \le {\dfrac 1 4}^2 \dfrac {\delta L} {\delta y}  w_5 w_4$$

И так далее

$$\dfrac {\delta L} {\delta x}  \le {\dfrac 1 4}^5 \dfrac {\delta L} {\delta y}  w_5 w_4 w_3 w_2 w_1$$

Таким образом, градиент начинает экспоненциально затухать, если веса маленькие.

Если веса большие - то градиент наоборот начнет экспоненциально возрастать - все взорвется

Для некоторых функций активации картина будет не столь катастрофична, но тоже неприятна.

В семинарском задании вы посмотрите, например, как ведет себя функция ReLU в этом случае



## Batch Normalization

### Плохой вариант борьбы с этим

Давайте на каждом слое просто нормировать каждый признак, используя среднее и дисперсию по батчу

$$\tilde{x}_i^j = \dfrac {x_i^j - Ex^j} {\sqrt{Dx^j}}$$




Проблема в том, что таким образом мы можем попасть в область линейной составляющей нашей функции. Например, в случае сигмоиды 

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lin_sigmoid.png" width="400">


Получаем набор линейных слоев фактически без функций активации -> все вырождается в однослойную сеть. Не то что нам надо

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/bn_relu.png" width="400">

В случае с ReLU подобной функцией активации, проблемы не исчезат. Если нормировать активации в интервал [0,1] то на каждом слое мы будем всегда обнулять половину активаций. 

Хотя опыт принудительной нормализации выходов слоя известен: в AlexNet было два слоя нормализации.

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/bn_loss.png" width="600">

Обратите внимание на то что в начале обучения Loss сильно меняется. В связи с этим происходят большие обновления весов в результате чего значтельная часть активаций может занулиться или уйти в область где Relu линейна.

На этом этапе нормализация могла бы помочь. Но по мере обучения сети изменения весов становятся более осмысленными.




Нам надо дать нейросетке возможность перемещать распределение слоя из области 0 и самой подбирать дисперсию 

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/nn_example.png" width="250">


$$\tilde{x}_i^j = \dfrac {x_i^j - Ex^j} {\sqrt{Dx^j}}$$



$$\hat{x_i^j} = \gamma^j\tilde{x}_i^j + \beta^j$$

Фактически, теперь нейросеть даже может отменить нормализацию, если считает ее ненужной

#### Скользящее среднее

Во время предсказания батча у нас уже нет - откуда брать матожидание и дисперсию? 

1. Можно посчитать по всей обучающей выборке - не всегда удобно, долго
2. Считать скользящее среднее во время обучения

$$E^j_{k+1} = (1 - \alpha)E^j_k + \alpha E^j_{batch}$$

Используем обычно $\alpha = 0.1$

Аналогично считаем дисперсию 

#### Защита от нулей в знаменателе

Чтобы у нас не мог возникнуть 0 в знаменателе, добавляем маленькое чиссло - $\epsilon$. Например, равное 1e-5


$$\tilde{x}_i^j = \dfrac {x_i^j - Ex^j} {\sqrt{Dx^j + \epsilon}}$$

$$\hat{x_i^j} = \gamma^j\tilde{x}_i^j + \beta^j$$

#### Линейный слои и конволюции

Слой конволюции можно свести к линейному слою с очень жесткими ограничениями на веса. Поэтому неудивительно, что BatchNorm можно применять и для линейных слоев и для конволюций. 

С конволюциями есть единственный нюанс - у нас "одним признаком" считается вся получаемая **feature map**. 

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/feature_map.png" width="450">

И нормализация идет по всей такой feature map (по всему каналу) для всех объектов. 

#### Пример работы

Этот метод действительно работает. 
Видим, что нейросети с батчнормализацией:

1. Сходятся быстрее, чем нейросети без 
2. Могут работать с более высоким начальным learning rate, причем это позволяет достигать лучших результатов
3. BatchNorm позволяет глубокой нейросетке работать даже с функцией активации в виде сигмоиды. Без батчнорма такая сеть не обучилась бы вовсе. 

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/batchnorm_work.png" width="500">


#### Градиент

Вычисление градиента batchnorm - интересное упражнение на понимание того, как работает backpropagation. В лекции мы это опускаем, можете ознакомиться самостоятельно

[Вывод градиентов для весов слоя BatchNorm](https://kevinzakka.github.io/2016/09/14/batch_normalization/)



<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-20.png" width="700">



<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gan/bn_graph.png" width="700">

https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html

### Covariate shift

Covariate shift - явление, когда признаки тренировочной выборки и тестовой по-разному распределены.

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/covariate_shift.png" width="300">

В такой ситуации модель не в состоянии делать адекватные предсказания на тесте, так как не видела области пространства, в которой расположены тестовые объекты

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/covariate_shift2.png" width="300">

### Internal covariate shift

Похожее явление может иметь место уже внутри нейросети

Пусть у нас iй слой переводит выдачу i-1 в новое пространство. 

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/internal_covariate_shift.png" width="400">

В конце нейросеть делает предсказание, считается лосс, делается обратное распространение ошибки и обновляются веса. 

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/internal_covariate_shift2.png" width="400">

После этого возникает нехорошая ситуация - распределение выходов i-1 слоя поменялось, а i-й слой изменял веса, думая, что распределение выходов не изменилось

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/internal_covariate_shift3.png" width="400">

### Internal covariate shift?

Согласно некоторым исследованиям ([например](https://arxiv.org/abs/1805.11604)), успех BatchNormalization не заключается в исправлении covariate shift. 

BatchNormalization работает как-то иначе, улучшая гладкость пространства решений и облегчает поиск в нем минимума.

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/batchnorm_smooth.png" width="700">

### Tips

Стоит помнить, что с батч-нормализацией:

* **Крайне важно** перемешивать батчи между эпохами. Единицей обучения параметров $\beta$ и $\gamma$ являются батчи. Если их не перемешивать, то из 6400 объектов в тренировочном датасете получим лишь 100 объектов для обучения $\beta$ и $\gamma$

* В слое, после которого поставили BatchNormalization, надо убрать смещения (параметр $\beta$ в BatchNormalization берет эту роль сам по себе)


* Другое расписание learning rate: бОльшее значение в начале обучения и быстрое уменьшение в процессе обучения

* Если используем BatchNormalization, то надо уменьшить силу Dropout и L2-регуляризации

* Чем меньше размер батча в обучении, тем хуже будет работать BatchNormalization

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/batchnorm_batch.png" width="400">


### Используем BatchNormalization



In [ ]:
class SimpleMNIST_NN_Init_Batchnorm(nn.Module):
    def __init__(self, n_layers):
        super().__init__()
        self.n_layers = n_layers
        layers = [nn.Linear(28 * 28, 100, bias=False),
                  nn.BatchNorm1d(100),
                  nn.Sigmoid()]
        for _ in range(0, n_layers - 1):
            layers.append(nn.Linear(100, 100, bias=False))
            layers.append(nn.BatchNorm1d(100))
            layers.append(nn.Sigmoid())
        layers.append(nn.Linear(100, 10))
        self.layers = nn.Sequential(*layers)
        self.init()
        
    def forward(self, x, predict_proba=False):
        x = x.view(-1, 28 * 28)
        x = self.layers(x)
        if predict_proba:
            x = F.softmax(x)
        return x 

    def init(self):
        sigmoid_gain = torch.nn.init.calculate_gain("sigmoid")
        for child in self.layers.children():
            if isinstance(child, nn.Linear):
                torch.nn.init.xavier_normal_(child.weight, 
                                              gain=sigmoid_gain)
                if child.bias is not None:
                    torch.nn.init.zeros_(child.bias)

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).cuda()
optimizer = optim.SGD(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss().cuda()
writer = get_summary_writer("batchnorm2")
train_steper = Steper()
test_steper = Steper()


In [ ]:
register_model_hooks(model, writer)

In [ ]:
for epoch in range(10):
    model.train()
    train_epoch(model,
              optimizer, 
              criterion,
              train_loader,
              "cross_entropy",
              train_steper,
              writer)
    model.eval()
    validate(model, 
           test_loader, 
           "cross_entropy", 
           test_steper,
           writer)

И попробуем, согласно советам, увеличить learning rate

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).cuda()
optimizer = optim.SGD(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss().cuda()
writer = get_summary_writer("batchnorm_increased_lr")

train_steper = Steper()
test_steper = Steper()


In [ ]:
register_model_hooks(model, writer)

In [ ]:
for epoch in range(10):
    model.train()
    train_epoch(model,
              optimizer, 
              criterion,
              train_loader,
              "cross_entropy",
              train_steper,
              writer)
    model.eval()
    validate(model, 
           test_loader, 
           "cross_entropy", 
           test_steper,
           writer)

In [ ]:
reinit_tensorboard(clear_log=False)

### Другие виды Нормализаций

[Множество их](https://paperswithcode.com/methods/category/normalization)

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/normalization_methods.png" width="900">



Чаще всего нормализации тестируют на изображениях в том числе или только на изображениях. Потому их графически объясняют при помощи следующей схемы:

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/notation_3d_tensor_viz.png" width="350">

По одной оси - каналы (feature maps), по второй - объекты из батча, а по третье feature map, reshapeнутого в одномерный вектор 



В этой терминологии BatchNorm выглядит следующим образом

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/batch_norm_3d.png" width="350">

#### Layer Norm

Нормализуем активации не вдоль объектов, а вдоль признаков, для каждого объекта индивидуально. Хорошо работает для рекуррентных нейронных сетей


<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/layer_norm_3d.png" width="350">


#### Instance Norm

При пакетной нормализации происходит усреднение параметров по всему пакету. Например, в случае задачи переноса стилей картин, это вносит много шума. При усреднении теряются индивидуальные характеристики объектов. Поэтому используется более тонкая нормализация — индивидуальная нормализация (англ. instance normalization) (по каналу, но только для данного объекта)

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/instance_norm_3d.png" width="350">

#### GroupNorm

Нормализуем активации по группам каналов (**feature_map**), как и в BatchNorm, но только для данного объекта.

Эффективен в случае, когда у нас батчи маленького размера (именно его мы сравнивали с обычной батч-нормализацией выше)

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/group_normalization_3d.png" width="350">



### Weights standartization 3d

Будем нормализовать **веса** каждого выходного слоя отдельно. 
Это в теории улучшает ландшафт, в котором мы ищем минимум функции потерь.
Обратите внимание, что здесь оси другие

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/weight_standartization3d.png" width="350">

За счет замены BatchNorm в популярной архитектуре ResNet на GroupNorm и Weights standartization была получена нейросеть, которую легко можно было адаптировать к разным задачам 

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/resnet_modified.gif" width="350">


### Ставить BatchNormalization до или после активации?

#### До

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/bn_act.png" width="350">


* Рекомендуется авторами статьи, где предложили Batch Normalization
* Для сигмоиды, BN, поставленная после активации, не решает проблем сигмоиды



#### После

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/act_bn.png" width="350">

* Аргументация авторов статьи не до конца обоснованна
* Обычно, сигмоиду не используют в современных нейронных сетях
* Для популярной ReLU, BN, поставленная до активации может приводить к “умирающей ReLU”, когда большая часть ее входов меньше 0 и потому для них градиент не проходит
* На многих задачах BN после функции активации работает лучше или не хуже поставленной до

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/bn_relu.png" width="500">

[ссылка](https://github.com/ducha-aiki/caffenet-benchmark/blob/master/batchnorm.md)

### Ставить BatchNormalization до или после Dropout?...

#### До
<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/bn_before.png" width="350">

* Меньше влияние (covariate shift) Dropout на Batchnorm

#### После

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/bn_after.png" width="350">

* Информация о зануленных активациях не просачивается через среднее и дисперсию батча

#### Ставить только что-то одно 

* Dropout может отрицательно влиять на качество нейросети с BatchNorm за счет разного поведения на train и test




#### Строго говоря

* Оптимальный порядок следования слоев зависит от задачи и архитектуры сети
* Возможно, стоит применять модифицированные версии BatchNorm

### Batchnorm как регуляризация

Почему для нейросети с батчнормализацией можно использовать более высокие learning rate? 

 

Оказывается, батчнормализация делает неявную регуляризацию на веса

## Регуляризация

Второй способ улучшения сходимости нейросетей и борьбы с переобучением - введение регуляризации. Ее можно вводить несколькими способами. 

### L1, L2 регуляризации

Самый простой способ мы уже разобрали - давайте просто добавим в лосс штраф к весам. На сходимость нейросети, это, правда, повлияет слабо

$$Loss\_reg = loss + \lambda \cdot reg$$

$$ reg_{L1} = \lambda \sum |w_i| $$

$$ reg_{L2} = \lambda \sum w_i^2 $$

<img src="https://edunet.kea.su/repo/src/L0X_Encoders/img/losses.gif" alt="alttext" width="500px"/>

Иногда уже его хватает, чтобы решить все проблемы. Напомним, что L2 лосс приводит к большому числу маленьких ненулевых весов в сети. А L1 лосс - к маленькому числу ненулевых весов (разреженной нейросети)

### Dropout

Одним из распространненных именно в нейросетях методом регуляризации является Dropout.

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-12.png" width="500">

Состоит этот метод в следующем:

1. Во время обучения мы с вероятностью *p* зануляем выход нейронов слоя (например, *p* = 0.5)
2. Зануленные нейроны не участвуют в данном forward, и градиент потому к ним при backward не идет. 

3. Сила регуляризации определяется вероятностью p, чем она больше - тем сильнее регуляризация. 


### Мотивация Dropout

#### Борьба с коадаптацией 

Одной из  проблем при работе с глубокими сетями является совместная адаптация нейронов. В такой сети, если все весовые коэффициенты изучаются вместе, обычно некоторые соединения будут иметь больше возможностей прогнозирования, чем другие.

Часть нейронов делает основную работу - предсказывает, а остальные могут вообще не вносить никакого вклада в итоговое предсказание. Или же другая картина - один нейрон делает кривоватое предсказание, другие его правят и в итоге первый нейрон своей ошибки не исправляет. 

Это явление называется со-адаптацией. Этого нельзя было предотвратить с помощью традиционной регуляризации, такой как L1 и L2. А вот Dropout с этим хорошо борется

Отмена способности некоторых нейронов учиться во время тренировки на самом деле оказывается направлена на то, чтобы получить более обученные нейроны и сократить переобучение, гарантирует, что выучиваемые индивидуальными нейронами функции хорошо работают со случайно выбранными подмножествами функций, выученных другими нейронами. 
Таким образом,  Dropout приводит к появлению более требовательных нейронов, которые пытаются обойти сложные, индивидуальные особенности, которые склонны плохо обобщать, и сохраняют больше полезной информации самостоятельно. 

На следующем рисунке (извлеченном из статьи Dropout: A Simple Way to Prevent Neural Networks from Overfitting) мы находим сравнение признаков, изученных в наборе данных MNIST с одним автоэнкодером скрытого слоя, имеющим 256 выпрямленных линейных единиц без отсева (слева), и признаков, изученных той же структурой с использованием отсева в ее скрытом слое с p=0,5 (справа).

В то время как первый показывает неструктурированные, беспорядочные паттерны, которые невозможно интерпретировать, второй явно демонстрирует целенаправленное распределение веса, которое обнаруживает штрихи, края и пятна самостоятельно, нарушая их взаимозависимость с другими нейронами для выполнения этой работы.

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lecture-13.png" width="600">

#### Dropout как регуляризация 

Фактически, Dropout штрафует слишком сложные, неустойчивые решения. Добавляя в нейросеть Dropout мы сообщаем ей о том, что решение, которое мы ожидаем, должно устойчиво к шуму

#### Dropout как ансамбль 

Можно рассматривать Dropout как ансамбль нейросетей со схожими параметрами, которые мы учим одновременно, вместо того, чтобы учить каждую в отдельности, а затем результат их предсказания усредняем, замораживая Dropout (http://mlg.eng.cam.ac.uk/yarin/blog_3d801aa532c1ce.html)

Фактически, возникает аналогия со случайным лесом - каждая из наших нейросетей легко выучивает выборку и переобучается - имеет низкий bias, но высокий variance. При этом за счет временного отключения активаций, каждая нейросеть видит не все объекты, а только часть. Усредняя все эти предсказания, мы уменьшаем variance. 



#### Dropout помогает бороться с переобучением

Dropout, в силу указанного выше, может хорошо помогать бороться с переобучением

И в случае линейных слоев 

<img src ="https://xuwd11.github.io/Dropout_Tutorial_in_PyTorch/index_files/index_22_0.png" width="600">


И в случае конволюций 


<img src ="https://xuwd11.github.io/Dropout_Tutorial_in_PyTorch/index_files/index_36_0.png" width="600">



#### Confidence interval от Dropout
Можно используя нейросеть с дропаутом, получить доверительный интервал для вашего предсказания. Просто не "замораживаем" dropout-слои во время предсказания, а делаем предсказания с активными dropout. 

И делаем forward через такую нейросеть для одного объекта 1000 раз. 
Сделав это 1000 раз вы получаете распределение предсказаний, на основе которого можно делать confidence интервалы и как раз ловить те объекты, на которых нейросетей вообще не понимает, что ей делать и потому предсказывает метку или еще что-то с сильной дисперсией. 



<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/confidence_nn.png" width="400">

###  Простая реализация Dropout выглядит следующим образом

In [ ]:
class BadDropout(nn.Module):
    def __init__(self, p: float = 0.5):
        super().__init__()
        if p < 0 or p > 1:
            raise ValueError("dropout probability has to be between 0 and 1, " "but got {}".format(p))
        self.p = p

    def forward(self, X):
        if self.training:
            keep = torch.rand(X.size()) > self.p
            if X.is_cuda:
                keep = keep.cuda()
            return X * keep
        return weights * (1 - self.p)  # in test time, expectation is calculated

Нежелательное свойство представленной выше схемы состоит в том, что мы должны масштабировать активации во время прогнозирования. Поскольку производительность тестирования критически важна, лучше всего использовать инвертированный dropout, который выполняет масштабирование во время обучения. Кроме того, если мы захотим убрать dropout из кода, функция прогнозирования останется без изменений.

In [ ]:
class Dropout(nn.Module):
    def __init__(self, p: float = 0.5):
        super().__init__()
        if p < 0 or p > 1:
            raise ValueError("dropout probability has to be between 0 and 1, " "but got {}".format(p))
        self.p = p

    def forward(self, X):
        if self.training:
            keep = torch.rand(X.size()) > self.p
            if X.is_cuda:
                keep = keep.cuda()
            return X * keep / (1 - self.p)
        return X # in test time, expectation is calculated intrinsically - we just not divide weights

Попробуем засунуть Dropout в нашу нейросеть

In [ ]:
class SimpleMNIST_NN_Dropout(nn.Module):
    def __init__(self, 
                 n_layers,
                 activation=nn.Sigmoid, 
                 init_form="normal"):
        super().__init__()
        self.n_layers = n_layers
        self.activation = activation()
        layers = [nn.Linear(28 * 28, 100), self.activation]
        for _ in range(0, n_layers - 1):
            layers.append(nn.Linear(100, 100))
            layers.append(Dropout())
            layers.append(self.activation)
        layers.append(nn.Linear(100, 10))
        self.layers = nn.Sequential(*layers)
        self.init_form = init_form
        if self.init_form is not None:
            self.init()
        if not isinstance(self.activation, nn.Sigmoid):
            raise NotImplementedError()
        
    def forward(self, x, predict_proba=False):
        x = x.view(-1, 28 * 28)
        x = self.layers(x)
        if predict_proba:
          x = F.softmax(x)
        return x 

    def init(self):
        sigmoid_gain = torch.nn.init.calculate_gain("sigmoid")
        for child in self.layers.children():
            if isinstance(child, nn.Linear):
                if self.init_form == "normal":
                  torch.nn.init.xavier_normal_(child.weight, 
                                                gain=sigmoid_gain)
                  if child.bias is not None:
                      torch.nn.init.zeros_(child.bias)
                elif self.init_form == "uniform":
                    torch.nn.init.xavier_uniform_(child.weight, 
                                                gain=sigmoid_gain)
                    if child.bias is not None:
                        torch.nn.init.zeros_(child.bias)
                else:
                    raise NotImplementedError()

In [ ]:
model = SimpleMNIST_NN_Dropout(n_layers=3).cuda()
optimizer = optim.SGD(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss().cuda()
writer = get_summary_writer("nn3_dropout")
train_steper = Steper()
test_steper = Steper()

Так как наша модель из-за Dropout ведет себя по-разному во время обучения и во время тестирования, то мы должны прямо ей сообщать, обучается она сейчас или нет. Делается это при помощи функций model.train и model.eval

In [ ]:
for epoch in range(10):
    model.train()
    train_epoch(model,
                  optimizer, 
                  criterion,
                  train_loader,
                  "cross_entropy",
                  train_steper,
                  writer)
    model.eval()
    validate(model, 
             test_loader, 
             "cross_entropy", 
             test_steper,
             writer)

В данном случае выигрыша мы не получили. Возможно, если учить нейросеть больше эпох, эффект бы заметили

#### Пример борьбы с переобучением при помощи Dropout
 

Чтобы увидеть эффект, и при этом не учить долго нейросеть 100+ эпох, сделаем искуственный пример.

Просто добавим к линейной зависимости шум и попробуем выучить ее нейронной сетью 

In [ ]:
#https://towardsdatascience.com/batch-normalization-and-dropout-in-neural-networks-explained-with-pytorch-47d7a8459bcd
N = 50 #number of data points
noise = 0.3

#generate the train data
X_train = torch.unsqueeze(torch.linspace(-1, 1, N),1)
Y_train = X_train + noise * torch.normal(torch.zeros(N,1), torch.ones(N,1))

#generate the test data
X_test = torch.unsqueeze(torch.linspace(-1,1,N),1)
Y_test = X_test + noise * torch.normal(torch.zeros(N,1), torch.ones(N,1))

In [ ]:
plt.scatter(X_train.data.numpy(), Y_train.data.numpy(), c='purple', alpha=0.5, label='train')
plt.scatter(X_test.data.numpy(), Y_test.data.numpy(), c='yellow', alpha=0.5, label='test')

x_real = np.arange(-1, 1, 0.01)
y_real = x_real
plt.plot(x_real, y_real, c="green", label='true')
plt.legend()
plt.show()

Модель без дропаут

In [ ]:
N_h = 100
model = torch.nn.Sequential(
    torch.nn.Linear(1, N_h),
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, N_h),
    torch.nn.ReLU(),
    torch.nn.Linear(N_h, 1),
)
opt = torch.optim.Adam(model.parameters(), lr=0.01)

Модель с дропаут

In [ ]:
N_h = 100

model_dropout = nn.Sequential(
    nn.Linear(1, N_h),
    nn.Dropout(0.5), #50 % probability 
    nn.ReLU(),
    torch.nn.Linear(N_h, N_h),
    nn.Dropout(0.2), #20% probability
    nn.ReLU(),
    torch.nn.Linear(N_h, 1),
)
opt_dropout = torch.optim.Adam(model_dropout.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()

In [ ]:
max_epochs = 1500

for epoch in range(max_epochs):
    
    pred = model(X_train) # look at the entire data in a single shot
    loss = loss_fn(pred, Y_train)
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    pred_dropout = model_dropout(X_train)
    loss_dropout = loss_fn(pred_dropout, Y_train)
    opt_dropout.zero_grad()
    loss_dropout.backward()
    opt_dropout.step()
    
    
    if epoch % 100 == 0:
        
        model.eval()
        model_dropout.eval()
        
        test_pred = model(X_test)
        test_loss = loss_fn(test_pred, Y_test)
        
        test_pred_dropout = model_dropout(X_test)
        test_loss_dropout = loss_fn(test_pred_dropout, Y_test)
        
        plt.scatter(X_train.data.numpy(), Y_train.data.numpy(), c='purple', alpha=0.5, label='train')
        plt.scatter(X_test.data.numpy(), Y_test.data.numpy(), c='yellow', alpha=0.5, label='test')
        plt.plot(X_test.data.numpy(), test_pred.data.numpy(), 'r-', lw=3, label='normal')
        plt.plot(X_test.data.numpy(), test_pred_dropout.data.numpy(), 'b--', lw=3,  label='dropout')
        
        plt.title('Epoch %d, Loss = %0.4f, Loss with dropout = %0.4f' % (epoch, test_loss, test_loss_dropout))
        
        plt.legend()

        model.train()
        model_dropout.train()
        
        plt.pause(0.05)

Видим, что нейросеть без dropout сильно переобучилась

### Dropconnect

Будем занулять не нейроны, а веса. Фактически - для каждого батча обрубаем часть связей между нейронами

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/dropconnect.png" width="700">

Drop Connect случайным образом отбрасывая веса, а не активации с вероятностью *p*.

DropConnect похож на Dropout, поскольку он вводит динамическую разреженность в модель , но отличается тем, что разреженность зависит от весов *W*, а не от выходных векторов слоя. Другими словами, полностью связанный слой с DropConnect становится разреженно связанным слоем, в котором соединения выбираются случайным образом на этапе обучения. 

В принципе, вариантов зануления чего-то в нейросетке можно предложить великое множество, в разных ситуациях будут работать разные ([в этом списке](https://paperswithcode.com/methods/category/regularization)  много Drop...)

### DropBlock

Например, можно убирать для каждого батча из нейросети случайные блоки из слоев. И это будет работать!

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/dropblock.png" width="700">

## Оптимизация весов нейросетей

Методов тоже много, расскажем о популярных ([неполный список](https://paperswithcode.com/methods/category/stochastic-optimization))

Эти методы реализованы в модуле torch.optim
При этом, что важно, подсчет градиентов лосс функции по весам никак не зависит от оптимизаторов из пакета. На прошлых занятиях мы с вами видели, что pytorch вычисляет градиенты автоматически, на основе оптимизаций

### SGD

Обычный стохастичный градиентный спуск. Вы его уже реализовывали. 
Просто обновляем веса в соответствии с текущем градиентом по ним, домножая градиент на постоянный коэффициент $\eta$





```
grad = evaluate_gradient(loss_fun,x,weights)
weights = weights - learning_rate * grad
```



<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/gradient_descent.gif" width="500">

До сих пор является достаточно популярным методом обучения нейросетей, ибо простой, не требует дополнительных параметров кроме $\eta$, и сам по себе обычно дает не плохие результаты. 

Если же важна каждая сотая в качестве, то нужно либо использовать его вкупе с другими технологиями (о них - ниже), либо использовать другие способы. 



In [ ]:
import torch
import torch.optim as optim

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.SGD([parameters], lr=0.001)

Минусы SGD:

 1. Если функция ошибки быстро меняется в одном направлении и медленно - в другом, то это приводит к резким изменениям направления  градиентов и замедляет процесс обучения

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/no-momentum.gif" width="500">


 2. Может застревать в локальных минимумах или седловых точках (точках, где все производные равны 0, но не являющихся минимума/ максимумами). В них градиент равен 0, веса не ообновляются - конец оптимизации. 

Пример таких точек:


Точка 0 у функции $x^3$, не имеющей минимума или максимума вовсе
<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/saddle_pointx3.png" width="400">

Или точка 0, 0 у функции y = $x^2 - y^2$

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/saddle_point_x2y2.png" width="400">

 3. Так как мы оцениваем градиенты по малой части выборки, они могут плохо отображать градиент по всей выборке и являться шумными. ВВ результате часть шагов градиентного спуска делаются впустую или во вред. 
 
 4. Мы применяем один и тот же learning rate ко всем параметрам, что не всегда разумно. Параметр, отвечающий реедкому классу, будет обучаться медленнее остальных. 
 
 5. Просто медленнее сходится

### Momentum

Чтобы избежать проблем 1-3, можно использовать momentum - фактически, мы добавляем нашему движению инерции. Если представить наши текущие веса как координаты шарика, и мы этот шарик пытаемся загнать в наиболее глубокую дырку, то у шарика теперь появился вес.

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/ball_1.gif" width="600">



```
vx = 0 # velocity
while True:
  dx = evaluate_gradient(loss_fun,x,weights)
  vx = vx*rho + dx # rho 0.9 .. 0.99 'friction'
  x -= lr *dx
```




Сначала пытаемся поменять направление движения шарик с прежнего направления с учетом текущего градиента
$$v_{t+1} = \rho v_t + \nabla_wL(x, y, W)$$

Вычисляем, куда он покатится
$$w_{t+1} = w_t - \eta v_{t+1}$$

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/momentum.png" width="400">

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.SGD([parameters], 
                      momentum=0.9,
                      lr=0.001)

Теперь мы быстрее достигаем локального минимуума и можем выкатываться из совсем неглубоких. Градиентн стал менее подвержен шум, меньше асциллирует

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/no-momentum.gif" width="400">
<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/momentum.gif" width="400">


У этого подхода есть одна опасность - мы можем выкатиться за пределы минимума, к которому стремимся, а потом какое-то время к нему возвращаться. 

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/increasing_momentum.gif" width="700">


Чтобы с этим бороться, предложен другой способ подсчета момента инерции

### NAG (Nesterov momentum)

Будем сначала смещаться в сторону, куда привел бы нас наш накопленный градиент, там считать новый градиент и смещаться по нему. 
В результате перескоки через минимум будут менее значительными и мы будем быстрее сходиться

$$v_{t+1} = \rho v_t +  \nabla_w L(w + \rho v_t )$$

$$w_{t} = w_{t-1} - \eta v_{t} $$

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/nesterov_momentum.png" width="800">

На практике эту формулу все равно можно записать так, чтобы задача вычисления градиента ложилась не на сам оптимизатор.
([реализация в pytorch](https://github.com/pytorch/pytorch/blob/master/torch/optim/_functional.py))

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.SGD([parameters],
                      momentum=0.9,
                      nesterov=True,
                      lr=0.001)

### Adaptive Learning Rate

Оба озвученных улучшения никак не борются с тем, что некоторые параметры нашей нейросети могут получать информацию для обновления реже, чем остальные. Например, фильтр, отличающий сиамских кошек от остальных, будет работать только тогда, когда в выборке попадутся конкретно сиамские кошки. А их может быть и не очень много, по батчам мы их не размажем (либо батчей поолучится очень много, если нарошно балансировать)

Другая ситуация - учим нейросеть на словах из русского языка. Есть веса, отвечающие за редкие слова, например "молвить". Вы часто в языке встречаете слово молвить? Молвят устами, а они тоже встречаются не часто. В результате, если learning rate постояннен, то мы выучим параметры для слова молвить плохо. 

Единственный путь - завести для каждого параметра индивидуальный learning rate.

### Adagrad

Будем хранить для каждого параметра сумму квадратов его градиентов. 

И будем добавлять наш градиент с коэффициентом, обратно пропорциональным корню из это суммы (фактически - некая аппроксимация средней амплитуды градиента нашего параметра). 

$$ w = w - \eta \frac{l}{\sqrt{G} + e} \odot (\nabla_w L(x,y,W)) $$

$$ G = \sum_{t=1}^T \nabla_w L(x,y,w_t)^2 $$


```
grad_squared = 0 # it's tensor !
while True:
  dx = evaluate_gradient(loss_fun,x,weights)
  grad_squared += dx*dx
  x = lr*dx / (sqrt(grad_squared) + 1e-7)
```  

В результате, если градиент у нашего веса часто большой - коэффициент будет уменьшаться. 

Единственная проблема - при такой формуле наш learning rate неминуемо в конце концов затухает (так как сумма квадратов не убывает)


In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.Adagrad([parameters])

### RMSprop

Давайте устроим "забывание" предыдущих квадратов градиентов. Просто будем домножать их на некий коэффициент меньше 0


$$v_t = \alpha v_{t-1} + (1-\alpha) (\nabla_w L(x,y,w_t))^2$$

$$w = w - \frac{\eta}{\sqrt{v_t }+ e} \odot \nabla_w L(x,y,W)$$


```
grad_squared = 0
decay_rate =0.9
while True:
  dx = evaluate_gradient(loss_fun,x,weights)
  grad_squared += decay_rate*grad_squared + (1-decay_rate)dx*dx
  x = lr*dx / (sqrt(grad_squared) + 1e-7)
```  


In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.RMSprop([parameters], alpha=0.99)

### Adam

Один из самых популярных адаптивных оптимизаторов является Adam. 
Получается он за счет объединения идеи с инерцией и идеи с суммой квадратов. 

$$ m_t = \beta_1 m_{t-1} + (1-\beta_1) (\nabla_w L(x,y,w_t)) $$
$$ v_t = \beta_2 v_{t-1} + (1-\beta_2) (\nabla_w L(x,y,w_t)^2) $$
$$ w = w - \eta \cdot \frac{m_t}{\sqrt{v_t} + e} $$





Чтобы в начале у нас получались очень большие шаги, будем дополнительно модицифировать инерцию и сумму квадратов

$$ m_t = \frac{m_t}{1-\beta_1^t} $$

$$ v_t = \frac{v_t}{1-\beta_2^t} $$

Псевдокод:

```
beta1 = 0.9
beta2 =0.9

first_momentum = 0
second_momentum = 0
while True:
  dx = evaluate_gradient(loss_fun,x,weights)
  first_momentum = first_momentum * beta1 +  (1-beta1)*dx
  second_momentum = second_momentum*beta2 + 
  (1-beta2)*dx*dx
  x = lr*first_momentum / (sqrt(second_momentum) + 1e-7)
```  

Реальный Pytorch код

In [ ]:
parameters = torch.randn(10, requires_grad=True)
optimizer = optim.Adam([parameters], 
                       betas=(0.9, 0.999))

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).cuda()
optimizer = optim.Adam(model.parameters(),
                       lr=1e-2)
criterion = nn.CrossEntropyLoss().cuda()
writer = get_summary_writer("adam")
train_steper = Steper()
test_steper = Steper()


In [ ]:
for epoch in range(10):
    model.train()
    train_epoch(model,
              optimizer, 
              criterion,
              train_loader,
              "cross_entropy",
              train_steper,
              writer)
    model.eval()
    validate(model, 
           test_loader, 
           "cross_entropy", 
           test_steper,
           writer)

### L2-loss

Все оптимизаторы так же поддерживают возможность добавления к ним напрямую L2-loss, коэффициент перед этим лоссом -  $\textrm{weight_decay}$

In [ ]:
parameters = torch.randn(10, 
                         requires_grad=True)
optimizer = optim.RMSprop([parameters], 
                          alpha=0.99, 
                          weight_decay=0.001)

Нюанс в том, что в Adam L2-loss учитывается не совсем верно. Потому есть поправленная версия Adam - AdamW. Но не факт, что она всегда лучше работает

### Сравнение оптимизаторов 

У каждого из предложенных оптимизаторов есть минусы и плюсы 



#### Методы с инерцией сходятся к решению более плавно, но могут "перелетать"

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/opt_smooth.gif" width="250">


#### Методы с адаптивным learning rate быстрее сходятся, более стабильны и меньше случайно блуждают

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/adaptive_stability.gif" width="250">


####  Алгоритмы без адаптивного learning rate сложнее выбираются из локальных минимумом

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/local_minima.gif" width="450">



#### Алгоритмы с инерцией осцилируют в седловых точках прежде чем найти верный путь

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/saddle_point.gif" width="450">



## Режимы обучения

Нам не обязательно поддерживать один и тот же learning rate в течении всего обучения. Более того, для того же SGD есть гарантии, что если правильно подобрать схему уменьшения learning rate, он сойдется к глобальномуу оптимуму.


Мы можем менять learning rate по некоторым правилам.  



### Ранняя остановка

Можем использовать критерий ранней остановки - когда лосс на валидационной выборке не улучшается patience эпох, умножаем learning rate на некий factor

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/early_stopping.png" width="500">

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.1)
scheduler = ReduceLROnPlateau(optimizer, 
                              'min', 
                              factor=0.1,
                              patience = 5)

Применим к нашей модели

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss().cuda()
writer = get_summary_writer("reduce_lr_on_plateu")
train_steper = Steper()
test_steper = Steper()
scheduler = ReduceLROnPlateau(optimizer, 
                              'min', 
                              factor=0.1,
                              patience = 1)


In [ ]:
for epoch in range(20):
    model.train()
    train_epoch(model,
              optimizer, 
              criterion,
              train_loader,
              "cross_entropy",
              train_steper,
              writer)
    model.eval()
    val_loss = validate(model, 
           test_loader, 
           "cross_entropy", 
           test_steper,
           writer)
    scheduler.step(val_loss)

### Домножать learning rate на alpha каждую эпоху

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
                                            step_size=2,
                                            gamma=0.1)



### Cyclical learning schedule





Мы можем все время понижать learning rate, а делать это циклически - то понижать, то повышать. Делать это можно по-разному:


1. Постоянно оставлять одни и те же границы

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/cycle_lr1.png" width="350">

2. Уменьшать верхнюю границу в сколько-то раз

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/cycle_lr2.png" width="350">
<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/cycle_lr3.png" width="350">


Нюанс - здесь мы ОБЯЗАТЕЛЬНО должны хранить модель с лучшим качеством. Такая оптимизация не гарантирует, что в конце модель будет лучше, чем на каком-то промежуточном шаге

#### Подбираем границы learning rate



1. просто берем и либо делаем для каждого learning rate минимизацию одну эпоху (чтобы всю выборку увидеть), нейросетку для каждого значения learning rate инициализируем заново

Это долго

Потому  делаем "магию":

2. На каждое значение learning rate у нас будет лишь один батч, нейросетку для каждого learning rate не меняем. И это часто работает, так как нам нужна просто грубая прикидка

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/lr_selection.png" width="500">

Полученные значения (оптимальные границы) и используем в качестве высокого и низкого порога на learning rate.

In [ ]:
start_lr = 1e-8
end_lr = 10
lr_find_epochs = 2
steps = lr_find_epochs * len(train_loader)
smoothing = 0.05

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).cuda()
criterion = nn.CrossEntropyLoss().cuda()
train_steper = Steper()
test_steper = Steper()



In [ ]:
import math

In [ ]:
lrs = []
losses = []
optimizer = optim.SGD(model.parameters(), lr=1e-8)
lr_lambda = lambda x: math.exp(x * math.log(end_lr / start_lr) / (steps))
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

for epoch in range(lr_find_epochs):
    for batch in train_loader:
        optimizer.zero_grad()
        X, y = batch
        X, y = X.cuda(), y.cuda()
        pred = model(X)
        loss = criterion(pred, y)

        loss.backward()
        optimizer.step()
        scheduler.step()

        loss = loss.detach().cpu().numpy()
        if len(losses) > 1:
          loss = smoothing  * loss + (1 - smoothing) * losses[-1]
        losses.append(loss)
        lr_step = optimizer.state_dict()["param_groups"][0]["lr"]
        lrs.append(lr_step)

In [ ]:
plt.figure(figsize=(20,10))
plt.title("LR range selection")
plt.plot(np.log10(lrs), losses)
plt.xlabel("Learning rate (log10)")
plt.ylabel("Loss")

На этом графике нам нужен минимум. Это в районе 1e-1. Делим это число на 10. 
Нижняя граница, стало быть - 1e-2.

Снижение лосса начинается с learning rate в районе 1e-4. Его делим на 6

In [ ]:
min_lr = 1e-4 / 6
max_lr = 1e-2

Этот scheduler надоо применять после каждого батча. Потому перепишем train_epoch_sh

In [ ]:
def train_epoch_sh(model, 
                optimizer,
                scheduler,
                criterion, 
                train_loader, 
                model_name,
                steper, 
                writer):
    for batch in train_loader:
        optimizer.zero_grad()
        X, y = batch
        X, y = X.cuda(), y.cuda()
        pred = model(X)
        loss = criterion(pred, y)
        writer.add_scalar(f"{model_name}_train_loss", 
                      loss.cpu().detach().numpy(),
                      global_step= steper.global_step)
        loss.backward()
        optimizer.step()
        steper.step()
        scheduler.step()


#### Запускаем

In [ ]:
model = SimpleMNIST_NN_Init_Batchnorm(n_layers=3).cuda()
criterion = nn.CrossEntropyLoss().cuda()
train_steper = Steper()
test_steper = Steper()
writer = get_summary_writer("sgd_cycle_lr")
optimizer = optim.SGD(model.parameters(), 
                      lr=min_lr)

scheduler = optim.lr_scheduler.CyclicLR(optimizer,
                                        base_lr=min_lr, 
                                        max_lr=max_lr,
                                        mode="triangular") # first case


По хорошемуу, в коде ниже мы на каждой эпохе сохраняем лучшую модель. 

In [ ]:
for epoch in range(20):
    model.train()
    train_epoch_sh(model,
              optimizer, 
              scheduler,
              criterion,
               train_loader,
              "cross_entropy",
              train_steper,
              writer)
    model.eval()
    val_loss = validate(model, 
           test_loader, 
           "cross_entropy", 
           test_steper,
           writer)

## Neural Network WarmUp

Также, для достаточно больших нейронных практикуют следующую схему (**gradual warmup**, [изначальная статья](https://arxiv.org/pdf/1706.02677.pdf) ):

Поставить изначальный learning rate значительно ниже того, с которого мы обычно начинаем обучение. За несколько эпох, например, 5, довести learning rate от этого значения до требуемого. За счет наша нейросет лучше "адаптируется" к нашим данным. 

Также такой learning schedule позволяет адаптивным оптимизаторам лучше оценить значения learning rate для разных параметров

<img src ="http://edunet.kea.su/repo/src/L07_Batch_normalization/img/warmup.png" width="1000">

kn на картинке - это размер одного батча

### Взаимодействие learning schedule и адаптивного изменения learning rate

И то, и другое меняет learning rate, learning scheduler - глобально, а адаптивные оптимизаторы - для каждого веса отдельно 

Часто их применяют вместе, особенно в случае критерия ранней остановки и WarmUp

Однако в случае циклического режима обучения так делают не всегда. Дело в том, что одна из его задач - как раз избежать использования адаптивных оптимизаторов, требующих бОльше памяти и дополнительных вычислений. 

Однако никаких препятствий к использованию того же Adam в компании вместе с циклическим режимом обучения нет. В [исходной статье](https://arxiv.org/pdf/2004.02401.pdf) так делают. 

# Ссылки

[Оптимизаторы1](https://theaisummer.com/optimization/)

[Оптимизаторы2](https://mlfromscratch.com/optimizers-explained/#adam)

[Батч-нормализация](https://theaisummer.com/normalization/)

[Циклический learning rate](https://towardsdatascience.com/adaptive-and-cyclical-learning-rates-using-pytorch-2bf904d18dee)

[Разные функции активации, затухающие и взрывающщиеся градиенты и т.д](https://mlfromscratch.com/activation-functions-explained/#/)

[Визуализация разных оптимизаторов в ipynb, но на tensorflow](https://nbviewer.jupyter.org/github/ilguyi/optimizers.numpy/blob/master/optimizer.tf.all.opt.plot.ipynb)